In [360]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import input_file_name, row_number,when,col,min,max,unix_timestamp,rank, dense_rank,date_format,count,lead,lag,sum,expr,from_utc_timestamp,substring,abs,from_unixtime,to_date,greatest,to_timestamp,lit,concat,split,current_date, concat_ws,regexp_extract,lpad,first, last,date_add,hour,coalesce,floor,window,size
from functools import reduce
from pyspark.sql.window import Window
spark = SparkSession.builder.appName("MulRead").getOrCreate()

In [330]:
# Read all CSV files in the folder
alfomatic_path = '/Users/sushantshinde/workspaces/trvi/alfomatic8-9/*.csv'
windowSpec_csv_filename = Window.partitionBy('FileName').orderBy("FileName")
df_alf = spark.read.format("csv") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .load(alfomatic_path).withColumn("FileName", input_file_name()).withColumn("join_date", date_format(col("_time"), "yyyy-MM-dd HH:mm:ss").cast("timestamp"))\
    .withColumn("FileName", input_file_name())\
    .withColumn("file_id", dense_rank().over(Window.orderBy("FileName")))\
    .filter(col("join_date").cast("timestamp").between('2025-03-08T22:00:00','2025-03-09T23:59:59'))

In [331]:
# df_alf.printSchema()

In [332]:
df2_pls = spark.read.format("csv") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .load("/Users/sushantshinde/workspaces/trvi/pls08/*.csv").withColumn("join_date", date_format(col("_time"), "yyyy-MM-dd HH:mm:ss").cast("timestamp"))\
    # 

In [333]:
df2_block = spark.read.format("csv") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .load("/Users/sushantshinde/workspaces/trvi/blockformer8-9/*.csv").withColumn("join_date", date_format(col("_time"), "yyyy-MM-dd HH:mm:ss").cast("timestamp"))\
    .withColumnRenamed("B1D1.AI_D1B31","AI_D1B31").withColumnRenamed("B1D1.AI_D1B32","AI_D1B32").withColumnRenamed("B1D1.AI_D1B34","AI_D1B34")\
    .filter(col("join_date").cast("timestamp").between('2025-03-08T22:00:00','2025-03-09T22:00:00'))

In [334]:
#  Convert _time to timestamp
df2_block = df2_block.withColumn("_time", to_timestamp("_time"))

# Create a window to track changes
window = Window.orderBy("_time")

# Use lag to compare current and previous values
df2_block = df2_block.withColumn("prev_valueblock1", lag("IO_MI_B1D1C3cut").over(window))
df2_block = df2_block.withColumn("prev_prev_valueblock1", lag("IO_MI_B1D1C3cut", 3).over(window))


# Create a new column 'start_time' in the original DataFrame
df2_block = df2_block.withColumn(
    "end_time_block1",
    # when(
    #     (col("IO_MI_B1D1C3cut") == 1) & (col("prev_value").isNull() | (col("prev_value") == 0)),
    #     col("_time")
    # ).otherwise(None)
     when(
        (col("IO_MI_B1D1C3cut") == 0) & (col("prev_valueblock1") == 1) 
        & (col('prev_prev_valueblock1') == 1)
        ,
        col("_time")
    ).otherwise(None)
)

df2_block = df2_block.withColumn("prev_valueblock2", lag("IO_MI_B1D2C3cut").over(window))
df2_block = df2_block.withColumn("prev_prev_valueblock2", lag("IO_MI_B1D2C3cut", 3).over(window))


# Create a new column 'start_time' in the original DataFrame
df2_block = df2_block.withColumn(
    "end_time_block2",
    # when(
    #     (col("IO_MI_B1D1C3cut") == 1) & (col("prev_value").isNull() | (col("prev_value") == 0)),
    #     col("_time")
    # ).otherwise(None)
     when(
        (col("IO_MI_B1D2C3cut") == 0) & (col("prev_valueblock2") == 1 ) 
        & (col('prev_prev_valueblock2') == 1)
        ,
        col("_time")
    ).otherwise(None)
)
# Fill null values in 'start_time' column
# window_fill = Window.orderBy("_time").rowsBetween(Window.currentRow, Window.unboundedFollowing)
# df_with_start_time = df_with_start_time.withColumn(
#     "start_time",
#     last("start_time", True).over(window_fill)
# )

# Display the DataFrame with start times
# df2_block.show(100)


In [335]:
df2_block = df2_block.withColumn('BlockEndTIme',
                                   when(col('end_time_block1').isNotNull(),col('end_time_block1'))
                                   .when(col('end_time_block2').isNotNull(),col('end_time_block2')).otherwise(None)
                                   )


df2_block = df2_block.withColumn("BlockEndTIme", col("BlockEndTIme").cast("timestamp"))
window_spec = Window.orderBy("BlockEndTIme")

df2_block = df2_block.withColumn("next_datetime", lead("BlockEndTIme").over(window_spec))

df2_block = df2_block.withColumn("time_diff_hours", 
                   when(
                       col("next_datetime").isNotNull(), 
                       (unix_timestamp(col("next_datetime")) - unix_timestamp(col("BlockEndTIme"))) / 3600
                   ).otherwise(0)  
)

df2_block = df2_block.withColumn("is_new_block", when(col("time_diff_hours") > 3, 1).otherwise(0))
window_spec_cumsum = Window.orderBy("BlockEndTIme").rowsBetween(Window.unboundedPreceding, Window.currentRow)

df2_block = df2_block.withColumn("block_id", sum("is_new_block").over(window_spec_cumsum) + 1)
# df = df.withColumn("batch_id", sum("is_new_batch").over(window_spec_cumsum) + 1)


df2_block =  df2_block.withColumn('BlockFomerId',
                                   when(col('end_time_block1').isNotNull(),'BlockFormer1')
                                   .when(col('end_time_block2').isNotNull(),'BlockFormer2').otherwise(None)
                                   )
df2_block = df2_block.withColumn("block_id", 
                   when(
                       col("is_new_block") == 1, 
                        col("block_id") - 1  # Assign the previous block_id to the first row of the new block
                   ).otherwise(col("block_id"))
)
# .filter(col('block_id') == 2)
df2_block = df2_block.drop("next_datetime", "time_diff_hours", "is_new_block")

df2_block = df2_block.filter(col('BlockEndTime').isNotNull())


In [336]:
def generate_block_ids(df):
    # Define a window ordered by BlockEndTime
    window_spec = Window.orderBy("BlockEndTime")

    # Assign IDs where BlockEndTime is not null, else keep null
    df = df.withColumn("block_former_id", when(col("BlockEndTime").isNotNull(), row_number().over(window_spec) - 1))

    return df

In [337]:
df2_block = generate_block_ids(df2_block)

In [338]:
df_block_group = df2_block.groupBy('block_former_id').agg(max('BlockEndTIme').alias('End time in Blockformer'),max('BlockFomerId').alias('which block is'))

In [339]:
final_join = df_alf.alias('df').join(df2_pls.alias('df2'), on="join_date", how="left").select('df.*','df2.Report_Belt1_Speed','df2.Report_Belt2_Speed','df2.Report_Belt2LanceFlow')

In [340]:
print(df2_block.columns)


['result', 'table', '_start', '_stop', '_time', '_measurement', 'AI_D1B31', 'AI_D1B32', 'AI_D1B34', 'IO_MI_B1D1C3cut', 'IO_MI_B1D2C3cut', 'join_date', 'prev_valueblock1', 'prev_prev_valueblock1', 'end_time_block1', 'prev_valueblock2', 'prev_prev_valueblock2', 'end_time_block2', 'BlockEndTIme', 'block_id', 'BlockFomerId', 'block_former_id']


In [341]:
# final_join.show(100,truncate=False)
vat_columns = [f"VatNo{i}" for i in range(1, 8)]

filter_conditions = [col(vat).isNotNull() for vat in vat_columns]


In [342]:
df_ids = final_join.withColumn('VatNo1',when((col('V_G20EMP_N') == 1) | (col('V_G20EMP_N_1') == 1), col('_time')))\
                .withColumn('VatNo2',when((col('V_G20EMP_N') == 2) | (col('V_G20EMP_N_1') == 2), col('_time')))\
                .withColumn('VatNo3',when((col('V_G20EMP_N') == 3) | (col('V_G20EMP_N_1') == 3), col('_time')))\
                .withColumn('VatNo4',when((col('V_G20EMP_N') == 4) | (col('V_G20EMP_N_1') == 4), col('_time')))\
                .withColumn('VatNo5',when((col('V_G20EMP_N') == 5) | (col('V_G20EMP_N_1') == 5), col('_time')))\
                .withColumn('VatNo6',when((col('V_G20EMP_N') == 6) | (col('V_G20EMP_N_1') == 6), col('_time')))\
                .withColumn('VatNo7',when((col('V_G20EMP_N') == 7) | (col('V_G20EMP_N_1') == 7), col('_time')))\
                

In [343]:
df_ids = df_ids.withColumn('VATEndTIme',
                                   when(col('VatNo1').isNotNull(),col('VatNo1'))
                                   .when(col('VatNo2').isNotNull(),col('VatNo2'))
                                   .when(col('VatNo3').isNotNull(),col('VatNo3'))
                                   .when(col('VatNo4').isNotNull(),col('VatNo4'))
                                   .when(col('VatNo5').isNotNull(),col('VatNo5'))
                                   .when(col('VatNo6').isNotNull(),col('VatNo6'))
                                   .when(col('VatNo7').isNotNull(),col('VatNo7'))
                                   .otherwise(None)
                                   )

In [344]:
# vat_columns = [f"Vat{i}" for i in range(1, 8)]

df_ids = df_ids.filter(reduce(lambda x, y: x | y, filter_conditions))

window_spec = Window.orderBy("_time")
df_ids = df_ids.withColumn("row_num", row_number().over(window_spec))


In [345]:
df_ids = df_ids.withColumn("upprz-betlbatch_id", floor((col("row_num") - 1) / 30))
df_ids = df_ids.withColumn("time_window", date_format(col("_time"), "HHmmss"))

vat_columns = [f"VatNo{i}" for i in range(1, 8)]  # vat_1_date to vat_7_date
df2 = df_ids.withColumn("id", 
                   concat_ws("", 
                             *[when(col(vat).isNotNull(), lit(f"vat{i}_"))
                             .otherwise(lit("")) for i, vat in enumerate(vat_columns, 1)],
                             col("time_window")
                   ))


# df_ids = df_ids.withColumn(
#     "id",
#     when(
#         # Check all VAT columns are non-null
#         greatest(*[col(vat).isNotNull() for vat in vat_columns]),
#         concat_ws("",
#             *[lit(f"vat{i}_") for i, vat in enumerate(vat_columns, 1)],
#             date_format("_time", "HHmmss")
#         )
#     ).otherwise(lit(None))  # Set to null if any VAT is null
# )

In [346]:
# Remove empty strings from the ID
df2 = df2.withColumn("id", 
                   concat_ws("", 
                             *[col("id").substr(i, 4) for i in range(1, 29, 4)]
                   ))

df2 = df2.withColumn("upper_belt_id", concat_ws("_", col("id"), col("upprz-betlbatch_id")))


In [347]:
# # Calculate lower belt start time (30 minutes after upper belt start time)
df2 = df2.withColumn("lower_belt_time", expr("cast(_time as timestamp) + INTERVAL 30 MINUTES"))



In [348]:
df_group = df2.groupBy('upprz-betlbatch_id').agg(min('VATEndTIme').alias('upper_start_time_first_curd'),max('VATEndTIme').alias('Upper_start_time_last_curd'),min('lower_belt_time').alias('end_time_first_curd'),max('lower_belt_time').alias('upper_end_time_last_curd'),min('upper_belt_id').alias('first_curd_id'),max('upper_belt_id').alias('last_curd_id'))

In [349]:
# df_group.show(df_group.count(),truncate=False)

In [350]:
# def generate_ids(df):
#     df = df.withColumn("time_part", date_format(col("lower_belt_time"), "HHmmss"))
#     df = df.withColumn("interval_id", (unix_timestamp(col("lower_belt_time")) - unix_timestamp(min(col("lower_belt_time")).over(Window.orderBy(col("lower_belt_time"))))) / 15)
#     # df = df.withColumn("vat_1", lit("Vat1"))
#     # df = df.withColumn("vat_1", split(col("upper_belt_id"), "_").getItem(0))
#     df = df.withColumn("vat_array", split(col("upper_belt_id"), "_"))

#     df = df.withColumn("vat_1", expr("filter(vat_array, x -> NOT x rlike '^[0-9]+$')"))
#     # df = df.withColumn("batch_id", col("interval_id").cast("int"))
#     df = df.withColumn("batch_id", split(col("upper_belt_id"), "_").getItem(2))
    
#     # Create the ID
#     df = df.withColumn("lower_belt_id", concat_ws("_", col("vat_1"), col("time_part"), col("batch_id")))
    
#     # To ensure the ID is the same for the next 15 seconds, 
#     # we can use the interval_id to group rows and assign the ID of the first row in each group
#     window_spec = Window.partitionBy("interval_id").orderBy("lower_belt_time")
#     df = df.withColumn("lower_belt_id", first("lower_belt_id").over(window_spec))

#     df = df.drop("vat_1", "batch_id",'time_part','interval_id','lower_belt_window','vat_array')
    
#     return df

In [351]:



def generate_lower_belt_ids(df):
    # Extract the timestamp in "HHmmss" format for lower belt
    df = df.withColumn("time_part", date_format(col("lower_belt_time"), "HHmmss"))

    # Compute interval_id based on 15-second intervals
    min_time = df.select(min("lower_belt_time")).collect()[0][0]
    df = df.withColumn("interval_id", floor((unix_timestamp(col("lower_belt_time")) - unix_timestamp(lit(min_time))) / 15))

    # Extract the VAT part (first part before the first underscore)
    df = df.withColumn("vat_prefix", split(col("upper_belt_id"), "_")[0])

    # Extract upper belt number (last numeric part of upper_belt_id)
    # df = df.withColumn("upper_belt_number", split(col("upper_belt_id"), "_")[-1])
    df = df.withColumn("upper_belt_number", 
                     when(size(split(col("upper_belt_id"), "_")) > 1, split(col("upper_belt_id"), "_").getItem(size(split(col("upper_belt_id"), "_")) - 1))
                     .otherwise(None))

    # Assign batch_id: Convert 30s upper belt ID into two 15s lower belt blocks
    df = df.withColumn("batch_id", (col("interval_id") / 2).cast("int"))  # Group into 30s blocks (2x 15s)

    # Define Window partitioned by batch_id
    batch_window = Window.partitionBy("batch_id", "upper_belt_id")

    # Get start and stop times for each 15s batch
    df = df.withColumn("start_time", min("lower_belt_time").over(batch_window))
    df = df.withColumn("stop_time", max("lower_belt_time").over(batch_window))

    # Format start and stop times
    df = df.withColumn("start_time_part", date_format(col("start_time"), "HHmmss"))
    df = df.withColumn("stop_time_part", date_format(col("stop_time"), "HHmmss"))

    # Assign the lower_belt_id: First 15 seconds gets the ID, next 15 seconds is NULL
    df = df.withColumn(
        "lower_belt_id",
        when(col("interval_id") % 2 == 0,  # Assign ID only for the first 15 seconds
            concat_ws(" - ", 
                concat_ws("_", col("vat_prefix"), col("start_time_part"), col("upper_belt_number")), 
                concat_ws("_", col("vat_prefix"), col("stop_time_part"), col("upper_belt_number"))
            )
        ).otherwise(None)  # Set NULL for the next 15s
    )

    # Ensure consistency within each 15s batch
    df = df.withColumn("lower_belt_id", first("lower_belt_id", ignorenulls=True).over(batch_window))

    # Drop unnecessary columns
    df = df.drop("vat_prefix", "batch_id", "time_part", "interval_id", "start_time", "stop_time", "start_time_part", "stop_time_part",'upper_belt_number')

    return df






In [352]:
df2 = generate_lower_belt_ids(df2)

df2 = df2.withColumn("upper_belt_number", 
                     when(size(split(col("lower_belt_id"), "_")) > 1, split(col("lower_belt_id"), "_").getItem(size(split(col("lower_belt_id"), "_")) - 1))
                     .otherwise(None))



/Users/sushantshinde/workspaces/trvi/my_env/lib/python3.9/site-packages/pyspark/sql/column.py:460: FutureWarning: A column as 'key' in getItem is deprecated as of Spark 3.0, and will not be supported in the future release. Use `column[key]` or `column.key` syntax instead.
  warnings.warn(


In [353]:
# blob_path = '/Users/sushantshinde/workspaces/trvi/output'
# df2.repartition(1).orderBy('file_id') \
#     .write \
#     .mode("overwrite") \
#     .option("header", True) \
#     .csv(blob_path)

In [354]:
df2 = df2.withColumn("lower_belt_time_exit_time", expr("cast(lower_belt_time as timestamp) + INTERVAL 60 MINUTES"))


In [355]:
df_lower_belt = df2.groupBy('upper_belt_number').agg(min('lower_belt_time').alias('lower_start_time_first_curd'),max('lower_belt_time').alias('Lower_start_time_last_curd'),
                                                min('lower_belt_time_exit_time').alias('Lower_end_time_first_curd'),max('lower_belt_time_exit_time').alias('Lower_end_time_last_curd')).filter(col('upper_belt_number').isNotNull())

In [ ]:
# df_lower_belt.orderBy('upper_belt_number').show(df_lower_belt.count(),truncate=False)

# df_lower_belt.count()

25/03/12 14:54:26 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/03/12 14:54:26 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/03/12 14:54:27 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/03/12 14:54:27 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/03/12 14:54:27 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/03/12 14:54:27 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/03/12 1

+-----------------+---------------------------+--------------------------+-------------------------+------------------------+
|upper_belt_number|lower_start_time_first_curd|Lower_start_time_last_curd|Lower_end_time_first_curd|Lower_end_time_last_curd|
+-----------------+---------------------------+--------------------------+-------------------------+------------------------+
|0                |2025-03-08 23:56:31.074    |2025-03-08 23:56:45.068   |2025-03-09 00:56:31.074  |2025-03-09 00:56:45.068 |
|1                |2025-03-08 23:57:01.058    |2025-03-08 23:57:15.06    |2025-03-09 00:57:01.058  |2025-03-09 00:57:15.06  |
|10               |2025-03-09 00:01:31.059    |2025-03-09 00:01:45.062   |2025-03-09 01:01:31.059  |2025-03-09 01:01:45.062 |
|100              |2025-03-09 00:46:31.059    |2025-03-09 00:46:45.059   |2025-03-09 01:46:31.059  |2025-03-09 01:46:45.059 |
|1000             |2025-03-09 08:17:31.061    |2025-03-09 08:17:45.058   |2025-03-09 09:17:31.061  |2025-03-09 09:17:4

25/03/12 14:54:29 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/03/12 14:54:29 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/03/12 14:54:29 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/03/12 14:54:29 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/03/12 14:54:30 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/03/12 14:54:30 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


2326

In [356]:
df_join_upper_lower = df_group.alias('df1').join(df_lower_belt.alias('df2'),col('df1.upprz-betlbatch_id') == col('df2.upper_belt_number'),how='left')

In [357]:
final_result = df_join_upper_lower.alias('df1').join(df_block_group.alias('df2'),col('df1.upprz-betlbatch_id') == col('df2.block_former_id'),how='left')

In [358]:
final_result.show(100,truncate=False)

25/03/12 15:54:48 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/03/12 15:54:48 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/03/12 15:54:48 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/03/12 15:54:48 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/03/12 15:54:48 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/03/12 15:54:48 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/03/12 1

+------------------+---------------------------+--------------------------+-----------------------+------------------------+-------------------+-------------------+-----------------+---------------------------+--------------------------+-------------------------+------------------------+---------------+-----------------------+--------------+
|upprz-betlbatch_id|upper_start_time_first_curd|Upper_start_time_last_curd|end_time_first_curd    |upper_end_time_last_curd|first_curd_id      |last_curd_id       |upper_belt_number|lower_start_time_first_curd|Lower_start_time_last_curd|Lower_end_time_first_curd|Lower_end_time_last_curd|block_former_id|End time in Blockformer|which block is|
+------------------+---------------------------+--------------------------+-----------------------+------------------------+-------------------+-------------------+-----------------+---------------------------+--------------------------+-------------------------+------------------------+---------------+--------

In [ ]:
blob_path = '/Users/sushantshinde/workspaces/trvi/output'
final_result.repartition(1).orderBy('block_former_id') \
    .write \
    .mode("overwrite") \
    .option("header", True) \
    .csv(blob_path)

25/03/12 15:55:34 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/03/12 15:55:34 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/03/12 15:55:34 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/03/12 15:55:34 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/03/12 15:55:34 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/03/12 15:55:34 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/03/12 1

25/03/12 18:54:10 WARN TransportChannelHandler: Exception in connection from /10.53.30.186:63432
java.io.IOException: Operation timed out
	at java.base/sun.nio.ch.SocketDispatcher.read0(Native Method)
	at java.base/sun.nio.ch.SocketDispatcher.read(SocketDispatcher.java:47)
	at java.base/sun.nio.ch.IOUtil.readIntoNativeBuffer(IOUtil.java:340)
	at java.base/sun.nio.ch.IOUtil.read(IOUtil.java:294)
	at java.base/sun.nio.ch.IOUtil.read(IOUtil.java:269)
	at java.base/sun.nio.ch.SocketChannelImpl.implRead(SocketChannelImpl.java:426)
	at java.base/sun.nio.ch.SocketChannelImpl.read(SocketChannelImpl.java:493)
	at io.netty.buffer.PooledByteBuf.setBytes(PooledByteBuf.java:254)
	at io.netty.buffer.AbstractByteBuf.writeBytes(AbstractByteBuf.java:1132)
	at io.netty.channel.socket.nio.NioSocketChannel.doReadBytes(NioSocketChannel.java:357)
	at io.netty.channel.nio.AbstractNioByteChannel$NioByteUnsafe.read(AbstractNioByteChannel.java:151)
	at io.netty.channel.nio.NioEventLoop.processSelectedKey(NioEve

In [ ]:
# window_spec = Window.orderBy("_time")
# # df2 = df2.withColumn("row_num", row_number().over(window_spec)


In [ ]:
# df2 = df2.withColumn("block_former_exit_time", expr("cast(lower_belt_time_exit_time as timestamp) + INTERVAL 45 MINUTES"))\
#     .withColumn("join_date_block", date_format(col("block_former_exit_time"), "yyyy-MM-dd HH:mm:ss"))


In [ ]:
# df2 = df2.withColumn("block_1", 
#                    when(col("row_num") % 2 == 1, col("lower_belt_id"))  # Odd rows go to block_1
#                    .otherwise(None))

# df2 = df2.withColumn("block_2", 
#                    when(col("row_num") % 2 == 0, col("lower_belt_id"))  # Even rows go to block_2
#                    .otherwise(None))
# df2 = df2.drop("row_num")


In [ ]:
# result_join = df2.alias('df').join(df2_block.alias('df2'), col('df.join_date_block') == col('df2.join_date'), how="left")\
#     # .select('df.*','df2.AI_D1B31','df2.AI_D1B32','df2.AI_D1B34','df2.IO_MI_B1D1C3cut','df2.IO_MI_B1D2C3cut','df2.end_time_block1','df2.end_time_block2','df2.BlockEndTIme','df2.BlockFomerId').drop('join_date_block')

In [ ]:
# final_join = final_join.withColumn('BlockFomerId',
#                                    when(col('end_time_block1').isNotNull(),'BlockFormer1')
#                                    .when(col('end_time_block2').isNotNull(),'BlockFormer2').otherwise(None)
#                                    )

# df_group = final_join.groupBy('BlockFomerId','file_id','V_G20EMP_N').agg(count("*").alias('blocks count')).filter((col('BlockFomerId').isNotNull()) & (col('V_G20EMP_N') != 0))

25/03/12 12:31:18 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/03/12 12:31:18 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/03/12 12:31:18 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/03/12 12:31:18 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/03/12 12:31:18 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/03/12 12:31:18 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/03/12 1